In [1]:
import os
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
CHROMA_PATH = os.getenv('CHROMA_PATH', 'chroma_embeddings')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'local-rag')
TEXT_EMBEDDING_MODEL = os.getenv('TEXT_EMBEDDING_MODEL', 'nomic-embed-text')  # Default model
TEMP_FOLDER = os.getenv('TEMP_FOLDER', './_temp')
embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL, show_progress=True)


C:\Users\somaa\AppData\Local\Temp\ipykernel_16512\2241661458.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL, show_progress=True)


In [3]:
db = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=CHROMA_PATH,
        embedding_function=embedding
    )

C:\Users\somaa\AppData\Local\Temp\ipykernel_16512\454650042.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [4]:
LLM_MODEL = "llama3.2:3b-instruct-q5_K_S"


In [5]:
def get_prompt():
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI assistant. Generate five variations of the user's question to enhance document retrieval from a vector database. Also Provide alternatives separated by newlines.
        Original question: {question}""",
    )

    template = """ Answer the question based ONLY on the provided context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    return QUERY_PROMPT, prompt

In [6]:
def query(input):
    if input:
        # Initialize the language model
        llm = ChatOllama(model="llama3.2:3b-instruct-q5_K_S")

        # Get the vector database instance

        # Get the query and answer prompts
        QUERY_PROMPT, prompt = get_prompt()
        
        # Multi-document retriever to retrieve from multiple relevant documents
        retriever = MultiQueryRetriever.from_llm(
            retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            llm=llm,
            prompt=QUERY_PROMPT
        )

        # Processing chain
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        response = chain.invoke(input)
        return response

    return None

In [9]:
queries = [
    "What are the risk factors associated with Google and Tesla?"
]

In [10]:
for q in queries:
    print(f"Query: {q}")
    answer = query(q)
    print(f"Answer: {answer}")
    print("-" * 50)

Query: What are the risk factors associated with Google and Tesla?













llamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]

Answer: Based on the text, some of the risk factors associated with Alphabet Inc. (Google) and Tesla include:

**Alphabet Inc. (Google)**

1. Changes in social, political, economic, tax, and regulatory conditions or in laws and policies governing a wide range of topics.
2. Antitrust complaints filed by the U.S. Department of Justice and state Attorneys General.
3. Legislative proposals and pending litigation in the U.S., EU, and around the world that could diminish or eliminate safe harbor protection for websites and online platforms.
4. The Digital Markets Act and Digital Services Act in Europe.
5. Fines and changes to business practices due to regulatory actions.
6. Increased cost of doing business and limitations on pursuing certain business models or offering certain products or services.
7. Fluctuations in foreign exchange rates, particularly with regards to international revenues.

**Tesla**

There are no specific risk factors mentioned in the text for Tesla. However, it can be i